In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import keras
from keras.layers import GlobalAveragePooling2D,Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout,BatchNormalization
from tensorflow.keras.models import Model , load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity

from tensorflow import keras


import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
dataset_path = os.listdir('/kaggle/input/shop-dataset/Shop DataSet')

label_types = os.listdir('/kaggle/input/shop-dataset/Shop DataSet')
print (label_types)  

['non shop lifters', 'shop lifters']


In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/kaggle/input/shop-dataset/Shop DataSet' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/kaggle/input/shop-dataset/Shop DataSet' + '/' +item) + '/' + room))
    
# Build a dataframe        
dataSet_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(dataSet_df.head())
print(dataSet_df.tail())

                tag                                         video_name
0  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
1  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
2  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
3  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
4  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
              tag                                         video_name
850  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
851  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
852  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
853  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
854  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...


In [4]:
df = dataSet_df.loc[:,['video_name','tag']]
df
df.to_csv('dataSet.csv')

In [5]:
dataSet_df = pd.read_csv("dataSet.csv")


print(f"Total videos in dataSet: {len(dataSet_df)}")



dataSet_df.sample(10)

Total videos in dataSet: 855


,Unnamed: 0,video_name,tag
140,140,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
403,403,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
836,836,/kaggle/input/shop-dataset/Shop DataSet/shop l...,shop lifters
333,333,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
205,205,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
404,404,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
111,111,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
85,85,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
359,359,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
5,5,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters


In [6]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [7]:
def identity_block(X, f, filters, initializer=random_uniform):
    """
    Implementation of the identity block 
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    
    F1, F2, F3 = filters
    
    # Save the input value. 
    X_shortcut = X
    
    
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X) # Default axis
    X = Activation('relu')(X)
    
   
    
    
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    
    
    X =Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X =BatchNormalization(axis = 3)(X)
    
    X = Add()([X,X_shortcut])
    X =Activation('relu')(X)
   
    return X

In [8]:
def convolutional_block(X, f, filters, s = 2, initializer=glorot_uniform):
    """
    Implementation of the convolutional block 
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    
    F1, F2, F3 = filters
    
    X_shortcut = X


    
    
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut) 
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    
   

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [9]:
def ResNet50_feature_extractor(input_shape = (224, 224, 3), training=False):
    """

    Arguments:
    input_shape -- shape of the images of the dataset
    

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    X = convolutional_block(X, f=3, filters = [128,128,512], s=2 )
    
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f=3, filters = [256, 256, 1024], s=2 )

    
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    
    X = convolutional_block(X, f=3, filters = [512, 512, 2048], s=2 )
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

# fine tuning the model

    X = GlobalAveragePooling2D()(X)    
    # Create model
    model = Model(inputs=X_input, outputs=X, name="ResNet50_feature_extractor")

    return model

In [10]:
R = ResNet50_feature_extractor(input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [11]:
R.summary()

Model: "ResNet50_feature_extractor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 55, 55,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 55, 55,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 55, 55,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 55, 55,    │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 55, 55,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 55, 55,    │     16,640 │ activation_2[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 55, 55,    │     16,640 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │      1,024 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 55, 55,    │          0 │ batch_normalizat

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [10]:
# Load video frames with batching
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]  # Convert BGR to RGB
            frames.append(frame)

            if max_frames > 0 and len(frames) == max_frames:
                break
    finally:
        cap.release()
    
    return np.array(frames)

In [11]:
from tensorflow.keras.layers import Lambda

def build_feature_extractor():
    # Build ResNet50 as a feature extractor
    feature_extractor = ResNet50_feature_extractor(
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Define preprocessing layer using Lambda
    preprocess_input = keras.applications.resnet50.preprocess_input

    # Input layer
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))

    # Apply the preprocessing using Lambda layer
    preprocessed = Lambda(preprocess_input)(inputs)

    # Pass the preprocessed input to the ResNet50 feature extractor
    outputs = feature_extractor(preprocessed)
    
    # Return the complete model
    return keras.Model(inputs, outputs, name="feature_extractor")

# Create the feature extractor
feature_extractor = build_feature_extractor()

# Check the summary of the model
feature_extractor.summary()


Model: "feature_extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ResNet50_feature_extractor      │ (None, 2048)           │    23,587,712 │
│ (Functional)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [12]:

# Function to extract features in batches
def extract_features_from_video(video_path, batch_size=32, max_frames=0):
    frames = load_video(video_path, max_frames=max_frames)
    features = []
    
    # Process frames in batches
    for i in range(0, len(frames), batch_size):
        batch = frames[i:i + batch_size]
        batch_features = feature_extractor.predict(batch)
        features.append(batch_features)

    return np.concatenate(features, axis=0)

In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(dataSet_df["tag"]))
print(label_processor.get_vocabulary())

labels = dataSet_df["tag"].values
# Assuming 'tag' is the correct column name
labels = label_processor(dataSet_df["tag"].values).numpy()

labels

['non shop lifters', 'shop lifters']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir, batch_size=32):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    # Take all class labels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    # Convert class labels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")  # (num_samples, MAX_SEQ_LENGTH)
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")  # (num_samples, MAX_SEQ_LENGTH, NUM_FEATURES)

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames
        frames = load_video(os.path.join(root_dir, path))
        print(len(frames))
        video_length = frames.shape[0]

        # Initialize a mask for the current video
        length = min(MAX_SEQ_LENGTH, video_length)
        frame_masks[idx, :length] = 1  # Mark the valid frames in the mask

        # Extract features in batches
        for start in range(0, video_length, batch_size):
            end = min(start + batch_size, video_length)
            batch = frames[start:end]

            # Make predictions for the current batch
            batch_features = feature_extractor.predict(batch)

            # Assign features to the corresponding positions
            for j in range(len(batch_features)):
                if start + j < MAX_SEQ_LENGTH:
                    frame_features[idx, start + j, :] = batch_features[j]

    return (frame_features, frame_masks), labels

# Example usage
train_data, train_labels = prepare_all_videos(dataSet_df, "train")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"Train labels in train set: {train_labels.shape}")

In [ ]:
import pickle

with open('resnet_labels.pkl', 'wb') as file:
    pickle.dump(train_labels, file)

In [ ]:
with open('resnet_data.pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [15]:
import pickle


# Load from a file
with open('/kaggle/input/resnet-data/resnet_data.pkl', 'rb') as file:
    data = pickle.load(file)



In [16]:
print(len(data))

2


In [17]:
with open('/kaggle/input/resnet-labels/resnet_labels.pkl', 'rb') as file:
    labels = pickle.load(file)

print(len(labels))

855


In [18]:
from sklearn.model_selection import train_test_split

# Extract frame features, frame masks, and labels
frame_features, frame_masks = data  # train_data[0] and train_data[1]
labels = labels

# Split the data into training and test sets
(
    X_train_features, X_test_features, 
    X_train_masks, X_test_masks, 
    y_train, y_test
) = train_test_split(frame_features, frame_masks, labels, test_size=0.2, random_state=42)

# Now you have split the data:
print(f"Training frame features shape: {X_train_features.shape}")
print(f"Test frame features shape: {X_test_features.shape}")
print(f"Training frame masks shape: {X_train_masks.shape}")
print(f"Test frame masks shape: {X_test_masks.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")


Training frame features shape: (684, 20, 2048)
Test frame features shape: (171, 20, 2048)
Training frame masks shape: (684, 20)
Test frame masks shape: (171, 20)
Training labels shape: (684, 1)
Test labels shape: (171, 1)


In [19]:
print(f'Min value: {X_train_features.min()}, Max value: {X_train_features.max()}')


Min value: 0.0, Max value: 71.02835083007812


In [20]:
from sklearn.preprocessing import MinMaxScaler


# Get the shape
train_n_samples, train_max_length, train_n_features = X_train_features.shape
test_n_samples, test_max_length, test_n_features = X_test_features.shape
# Step 1: Reshape to 2D (combine samples and test_max_length)
train_features_reshaped = X_train_features.reshape(-1, train_n_features)  # Shape: (684*20, 2048)
test_features_reshaped = X_test_features.reshape(-1, test_n_features)  # Shape: (171*20, 2048)

In [21]:
# Step 2: Apply normalization
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_features = scaler.fit_transform(train_features_reshaped)
X_test_features = scaler.fit_transform(test_features_reshaped)

In [22]:
print(X_train_features)

[[0.         0.23940945 0.17483997 ... 0.30989122 0.24385214 0.40914583]
 [0.         0.23754597 0.17380619 ... 0.31367946 0.23992586 0.39996356]
 [0.         0.2414813  0.17313766 ... 0.31785083 0.24306631 0.39601147]
 ...
 [0.         0.3814845  0.3644786  ... 0.55502295 0.4540305  0.3107202 ]
 [0.         0.4221387  0.36453485 ... 0.59175634 0.43866634 0.29425305]
 [0.         0.4219632  0.36382675 ... 0.5914171  0.43835592 0.29178023]]


In [23]:
# Step 3: Reshape to 3D 
X_train_features = X_train_features.reshape(train_n_samples, train_max_length, train_n_features)
X_test_features = X_test_features.reshape(test_n_samples, test_max_length, test_n_features)


In [24]:
print(f'Original shape: {train_features_reshaped.shape}')
print(f'Reshaped back to: {X_train_features.shape}')

Original shape: (13680, 2048)
Reshaped back to: (684, 20, 2048)


In [25]:
print(X_train_features)

[[[0.         0.23940945 0.17483997 ... 0.30989122 0.24385214 0.40914583]
  [0.         0.23754597 0.17380619 ... 0.31367946 0.23992586 0.39996356]
  [0.         0.2414813  0.17313766 ... 0.31785083 0.24306631 0.39601147]
  ...
  [0.         0.25306797 0.1770773  ... 0.3063457  0.2443533  0.3907019 ]
  [0.         0.25206423 0.17783833 ... 0.31078672 0.24338102 0.3905288 ]
  [0.         0.24910212 0.17108631 ... 0.3015659  0.24362993 0.3931458 ]]

 [[0.         0.4559555  0.31021595 ... 0.5252514  0.45558405 0.23446794]
  [0.         0.48096132 0.3062091  ... 0.54097223 0.4560995  0.24714814]
  [0.         0.4896984  0.30679846 ... 0.5378914  0.47103882 0.2819677 ]
  ...
  [0.         0.4582138  0.30495024 ... 0.55336213 0.43742132 0.24372359]
  [0.         0.43711472 0.31272364 ... 0.5408187  0.44770813 0.3265623 ]
  [0.         0.43403864 0.31663465 ... 0.537204   0.45973778 0.32602173]]

 [[0.         0.14003038 0.23236561 ... 0.29556394 0.2695346  0.34716332]
  [0.         0.170521

In [38]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(32, return_sequences=True)(
        frame_features_input, mask=mask_input
    )

    
    x = keras.layers.GRU(8)(x)
#     x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
    )
    return rnn_model



In [39]:
seq_model = get_sequence_model()

In [40]:
seq_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 20, 2048)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_4 (GRU)         │ (None, 20, 32)    │    199,872 │ input_layer_6[0]… │
│                     │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_5 (GRU)         │ (None, 8)         │      1,008 │ gru_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 8)         │         72 │ gru_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 2)         │         18 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 200,970 (785.04 KB)

 Trainable params: 200,970 (785.04 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
def run_experiment():
    filepath = "video_classifier.keras"
    checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=filepath,  # File path where the model will be saved
    monitor='val_accuracy',  # Metric to monitor
    verbose=1,  # Verbosity mode, 1 will print messages when saving the model
    save_best_only=True,  # Save only the best model according to the monitored metric
    mode='max'  # Mode for the monitored metric, 'min' to save when it decreases
)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',
    patience=20,
    restore_best_weights=True
    )
    reduce_lr =  keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=5,
    min_lr=0.00000001,
    )
    

    
    history = seq_model.fit(
        [X_train_features, X_train_masks],
        y_train,
        validation_split=0.2,
        epochs=EPOCHS,
        callbacks=[checkpoint,early_stopping,reduce_lr]
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([X_test_features, X_test_masks], y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model
_, sequence_model = run_experiment()


Epoch 1/100
15/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5858 - loss: 0.7221
Epoch 1: val_accuracy improved from -inf to 0.67883, saving model to video_classifier.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.5869 - loss: 0.7149 - val_accuracy: 0.6788 - val_loss: 0.6574 - learning_rate: 0.0100
Epoch 2/100
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6397 - loss: 0.6675
Epoch 2: val_accuracy did not improve from 0.67883
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6368 - loss: 0.6676 - val_accuracy: 0.6788 - val_loss: 0.6281 - learning_rate: 0.0100
Epoch 3/100
16/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6099 - loss: 0.6458
Epoch 3: val_accuracy improved from 0.67883 to 0.89051, saving model to video_classifier.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6116 - loss: 0.6454 - val_accuracy: 0.8905 - val_loss: 0.6035 - learning_rate: 0.0100
Epoch 4/100
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7986 - loss: 0.5783
Ep

In [42]:
import random

# Function to randomly pick a frame from the video and predict its class
def predict_random_frame(video_path):
    # Load the video frames
    frames = load_video(video_path)

    # Randomly select a frame
    random_frame_index = random.randint(0, len(frames) - 1)
    random_frame = frames[random_frame_index]

    # Preprocess the random frame (resize, normalize, etc. if needed)
    random_frame = np.expand_dims(random_frame, axis=0)  # Add batch dimension
    random_frame_features = feature_extractor.predict(random_frame)  # Extract features
    random_frame_features = np.expand_dims(random_frame_features, axis=0)  # Add sequence dimension

    # Create a mask for the single frame
    random_frame_mask = np.zeros((1, MAX_SEQ_LENGTH), dtype="bool")
    random_frame_mask[0, 0] = 1  # Only the first position is valid

    # Pad the frame features to match the MAX_SEQ_LENGTH expected by the model
    padded_frame_features = np.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
    padded_frame_features[0, 0, :] = random_frame_features  # Insert the features

    # Predict the class using the trained model
    prediction = sequence_model.predict([padded_frame_features, random_frame_mask])
    
    # Get the predicted class
    predicted_class_index = np.argmax(prediction)
    class_vocab = label_processor.get_vocabulary()
    predicted_class = class_vocab[predicted_class_index]

    print(f"Predicted class for the random frame: {predicted_class}")

# Example usage with a video path
video_path = "/kaggle/input/shop-dataset/Shop DataSet/non shop lifters/shop_lifter_n_100.mp4"
predict_random_frame(video_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Predicted class for the random frame: non shop lifters


In [43]:
seq_model.save('model.h5')

In [44]:
seq_model.save_weights('model.weights.h5')